In [1]:
import json
import random

In [ ]:
# References:
# https://epic7x.com/equipment-tutorial/
# https://page.onstove.com/epicseven/global/view/7902683

In [128]:
# Equipment types
# main_stats: The main stats allowed on this equipment type
# sub_stats: The sub stats allowed on this equipment type

TYPES = {
    'Dagger': {
        'MainStats' : [0],
        'SubStats' : [1000, 1001, 1004, 1005, 1006]
    },
    'Ring': {
        'MainStats' : [0, 1],
        'SubStats' : [1000, 1001, 1002, 1003, 1004, 1005, 1006]
    }
}

In [1543]:
# Equipment Grades: White (Normal), Green (Good), Blue (Rare), Purple (Heroic), Red (Epic)

GRADES = {
    'Normal': {
        'starting_substats' : 0,
        'max_substats' : 4,
        'weight' : 0.05
    },
    'Good': {
        'starting_substats' : 1,
        'max_substats' : 4,
        'weight' : 0.1
    },
    'Rare': {
        'starting_substats' : 2,
        'max_substats' : 4,
        'weight' : 0.4
    },
    'Heroic': {
        'starting_substats' : 3,
        'max_substats' : 4,
        'weight' : 0.35
    },
    'Epic': {
        'starting_substats' : 4,
        'max_substats' : 4,
        'weight' : 0.1
    }
}

In [130]:
# Health – Gives +15% health (2 pieces)
# Defense – Gives +15% defense (2 pieces)
# Attack – Gives +35% attack (4 pieces)
# Speed – Gives 25% speed (4 pieces)
# Critical – Gives +12% critical rate (2 pieces)
# Hit Rate – Gives +20% effectiveness(2 pieces)
# Destruction – Gives +40% critical damage (4 pieces)
# Lifesteal – Gives +20% lifesteal of the damage dealt to enemies (4 pieces)
# Counter – Gives +20 chance to counter attack when attacked. (4 pieces)
# Resist – Gives +20% effect resistance(2 pieces)
# Unity – Gives +4% chance to trigger dual attack (2 pieces)
# Rage – Gives +30% damage when the enemy is debuffed (4 pieces)
# Immunity – Gives 1 turn immunity buff at the start of each battle phase (2 pieces)

SETS = {
    'Health': {
        'text' : 'Health – Gives +15% health (2 pieces)', # Text description of set 
        'items_req' : 2, # Number of items req to activate set
        'key_stat' : 'health_percent', # Stat that the set boosts
        'value' : 15 # Value by which the stat is boosted
    },
    'Defense': {
        'text' : 'Defense – Gives +15% defense (2 pieces)',
        'items_req' : 2, 
        'key_stat' : 'def_percent',
        'value' : 15
    },
    'Speed': {
        'text': 'Speed – Gives 25% speed (4 pieces)',
        'items_req': 4,
        'key_stat': 'speed_percent',
        'value': 25
    }
}

In [812]:
# Simplified: Main Stats and Sub Stats
# id: uniqu id for sta
# text: text description of the stat
# key_stat: the actual stat that the stat modified
# vars: # key: This goes in text, # type: fixed or random, # values : 3 values for the 3 different tiers (for now assume 3 different tiers of items)

STATS = {
    'MainStats': {
        '0' : {
            'id' : 0,
            'text': '<A>% Atack',
            'key_stat' : 'attack_percent',
            'vars' : [{'key': '<A>', 'type' : 'fixed', 'values' : [10, 11, 12]}],
            'upgrade_vals': [5, 5, 6, 6, 16]
        },
        '1' : {
            'id' : 1,
            'text': '<A> Atack',
            'key_stat' : 'attack_flat',
            'vars' : [{'key': '<A>', 'type' : 'fixed', 'values' : [50, 55, 60]}],
            'upgrade_vals': [10, 10, 20, 20, 50]
        }
    },
    'SubStats': {
        '1000' : {
            'id' : 1000,
            'text': '<A>% Atack',
            'key_stat' : 'attack_percent',
            'vars' : [{'key': '<A>', 'type' : 'rand', 'values' : [[1, 3], [2, 4], [3, 5]]}]
        },
        '1001' : {
            'id' : 1001,
            'text': '<A> Atack',
            'key_stat' : 'attack_flat',
            'vars' : [{'key': '<A>', 'type' : 'rand', 'values' : [[10, 15], [13, 18], [15, 20]]}]
        },
        '1002' : {
            'id' : 1002,
            'text': '<A>% Defense',
            'key_stat' : 'defense_percent',
            'vars' : [{'key': '<A>', 'type' : 'rand', 'values' : [[1, 3], [2, 4], [3, 5]]}]
        },
        '1003' : {
            'id' : 1003,
            'text': '<A> Defense',
            'key_stat' : 'defense_flat',
            'vars' : [{'key': '<A>', 'type' : 'rand', 'values' : [[10, 15], [13, 18], [15, 20]]}]
        },
        '1004' : {
            'id' : 1004,
            'text': '<A> Speed',
            'key_stat' : 'speed_flat',
            'vars' : [{'key': '<A>', 'type' : 'rand', 'values' : [[1, 3], [1, 4], [2, 5]]}]
        },
        '1005' : {
            'id' : 1005,
            'text': '<A>% Crit Rate Chance',
            'key_stat' : 'crit_rate',
            'vars' : [{'key': '<A>', 'type' : 'rand', 'values' : [[1, 3], [2, 4], [3, 6]]}]
        },
        '1006' : {
            'id' : 1006,
            'text': '<A>% Crit Damage',
            'key_stat' : 'crit_damage',
            'vars' : [{'key': '<A>', 'type' : 'rand', 'values' : [[10, 13], [12, 14], [13, 16]]}]
        }
    }
}

In [132]:
# Item Tiers, if item is Lv 1-10 it is Tier 1, if it is Lv 11-20 it is Tier 2, if it is Lv 21-30 it is Tier 3

TIERS = {
    'Tier 1': {
        'item_tier' : 1,
        'level_range' : range (1,11)
    },
    'Tier 2': {
        'item_tier' : 2,
        'level_range' : range (11,21)
    },
    'Tier 3': {
        'item_tier' : 3,
        'level_range' : range(21,31)
    }
}

False

In [9]:
# Check what we have so far:
list(TYPES), list(GRADES), list(SETS), list(STATS), list(TIERS)

(['Dagger', 'Ring'],
 ['Normal', 'Uncommon', 'Rare', 'Heroic', 'Epic'],
 ['Health', 'Defense', 'Speed'],
 ['MainStats', 'SubStats'],
 ['Tier 1', 'Tier 2', 'Tier 3'])

In [10]:
# Get a stat by particular id:

def get_stat_by_id(id):

	str_id = str(id) # Convert the provided ID to a string
	for s in STATS: # Iterate through the 'Main_Stats', 'Sub_Stats', etc. sections
		if str_id in STATS[s]: # # Check if the provided ID exists in the current section
			return STATS[s][str_id] # Return the stat dictionary associated with the ID
	return None # Return None if the provided ID doesn't match any stat

In [11]:
get_stat_by_id(1001)

{'id': 1001,
 'text': '<A> Atack',
 'key_stat': 'attack_flat',
 'vars': [{'key': '<A>',
   'type': 'rand',
   'values': [[10, 15], [13, 18], [15, 20]]}]}

In [115]:
list(set([]) | set(STATS['MainStats']))

['0', '1']

In [139]:
TYPES['Dagger']['SubStats']

[1000, 1001, 1004, 1005, 1006]

In [255]:
def get_random_stat(stat_type = 'MainStats', item_type = None):

    pool = [] # Initialize an empty list to store available IDs
    
    # If the provided stat_type is valid and exists in STATS
    if stat_type in STATS:
        # If no item type provided, pick any stat from list of main stats or substats
        if item_type is None:
            # Use set union to merge the available IDs into the pool list
            pool = list(set(STATS[stat_type]))
        else:
            pool = list(set(TYPES[item_type][stat_type])) # Get the pool of id's possible for this item_type
        
    # If the pool is empty, no stats are available, so return None
    if pool == []:
        return None

    # Choose a random ID from the pool and fetch the associated stat
    return get_stat_by_id(id = random.choice(pool))

In [352]:
get_random_stat('SubStats', 'Dagger')

{'id': 1000,
 'text': '<A>% Atack',
 'key_stat': 'attack_percent',
 'vars': [{'key': '<A>', 'type': 'rand', 'values': [[1, 3], [2, 4], [3, 5]]}]}

In [353]:
# This function gets a stat that hasn't already been selected
def get_non_overlapping_stat(selected = [], stat_type = 'MainStats', item_type = None):

	stat = get_random_stat(stat_type, item_type) # Get a random stat

    # If the stat we got in the previous line exists in our selected stats, get a new random stat
	while any(stat['id'] == a['id'] for a in selected): 
		stat = get_random_stat(stat_type, item_type) # Get a random stat
 
	return stat

In [440]:
# Get a random stat for an item
stat1 = get_random_stat('MainStats', 'Ring')
stat1

{'id': 0,
 'text': '<A>% Atack',
 'key_stat': 'attack_percent',
 'vars': [{'key': '<A>', 'type': 'fixed', 'values': [10, 11, 12]}]}

In [421]:
# Get a non overlapping stat (i.e. pick anything except for stat1)
get_non_overlapping_stat([stat1], 'SubStats', 'Dagger')

{'id': 1001,
 'text': '<A> Atack',
 'key_stat': 'attack_flat',
 'vars': [{'key': '<A>',
   'type': 'rand',
   'values': [[10, 15], [13, 18], [15, 20]]}]}

In [17]:
stat1['vars']

[{'key': '<A>', 'type': 'rand', 'values': [[1, 3], [2, 4], [3, 5]]}]

In [18]:
for var in stat1['vars']:
    print(var)

{'key': '<A>', 'type': 'rand', 'values': [[1, 3], [2, 4], [3, 5]]}


In [19]:
stat1['vars'][0]['values'][2]

[3, 5]

In [20]:
stat1

{'id': 1002,
 'text': '<A>% Defense',
 'key_stat': 'defense_percent',
 'vars': [{'key': '<A>', 'type': 'rand', 'values': [[1, 3], [2, 4], [3, 5]]}]}

In [21]:
def parse_stat(stat, tier = None):

    parsed_stat = {} # Initialize an empty dictionary to store parsed values
    parsed_stat['id'] = stat['id'] # Copy the 'id' from the input stat
    parsed_stat['text'] = stat['text'] # Copy the 'text' description from the input stat
    parsed_stat['key_stat'] = stat['key_stat'] # Copy the 'key_stat' from the input stat
    parsed_stat['values'] = [] # Initialize an empty list to store parsed values

    for var in stat['vars']: # Loop through the 'vars' list of the input stat
        key = var['key']
        type_ = var['type']
        values = var['values']

        if type_ == 'rand': # If the type is 'rand', calculate a random value within a range
            min_ = values[tier-1][0]
            max_ = values[tier-1][1]
            value = random.randint(int(min_), int(max_)) # Generate a random value

        # elif type_ == 'choice':
        # 	value = random.choice(values)
        elif type_ == 'fixed': # If the type is 'fixed', directly use the value for the tier
            value = values[tier-1]

        parsed_stat['values'].append([key, value]) # Append the parsed key-value pair to the 'values' list
    return parsed_stat

In [22]:
stat1_parsed = parse_stat(stat1, 1)

In [23]:
def format_stat(parsed_stat):

	text = parsed_stat['text'] # Get the original 'text' description from the parsed stat

    # Iterate through the key-value pairs in the 'values' list of the parsed stat
	for key, value in parsed_stat['values']:
		text = text.replace(key, str(value)) # Replace each occurrence of 'key' with 'value' in the text

    # Return the formatted text representation of the parsed stat
	return text

In [24]:
stat1_parsed = parse_stat(stat1, 1)
format_stat(stat1_parsed)

'1% Defense'

In [ ]:
# TO DO
# Add Item Upgrade Level
# Turn everything into an item class
# Create random item ()
# Create item with particular affixes


In [25]:
TIERS

{'Tier 1': {'item_tier': 1, 'level_range': range(1, 11)},
 'Tier 2': {'item_tier': 2, 'level_range': range(11, 21)},
 'Tier 3': {'item_tier': 3, 'level_range': range(21, 31)}}

In [26]:
for tier in TIERS:
    print (tier)

Tier 1
Tier 2
Tier 3


In [1542]:
# Check what we have so far:
list(TYPES), list(GRADES), list(SETS), list(STATS), list(TIERS)

(['Dagger', 'Ring'],
 ['Normal', 'Uncommon', 'Rare', 'Heroic', 'Epic'],
 ['Health', 'Defense', 'Speed'],
 ['MainStats', 'SubStats'],
 ['Tier 1', 'Tier 2', 'Tier 3'])

In [872]:
STATS

{'MainStats': {'0': {'id': 0,
   'text': '<A>% Atack',
   'key_stat': 'attack_percent',
   'vars': [{'key': '<A>', 'type': 'fixed', 'values': [10, 11, 12]}],
   'upgrade_vals': [5, 5, 6, 6, 16]},
  '1': {'id': 1,
   'text': '<A> Atack',
   'key_stat': 'attack_flat',
   'vars': [{'key': '<A>', 'type': 'fixed', 'values': [50, 55, 60]}],
   'upgrade_vals': [10, 10, 20, 20, 50]}},
 'SubStats': {'1000': {'id': 1000,
   'text': '<A>% Atack',
   'key_stat': 'attack_percent',
   'vars': [{'key': '<A>',
     'type': 'rand',
     'values': [[1, 3], [2, 4], [3, 5]]}]},
  '1001': {'id': 1001,
   'text': '<A> Atack',
   'key_stat': 'attack_flat',
   'vars': [{'key': '<A>',
     'type': 'rand',
     'values': [[10, 15], [13, 18], [15, 20]]}]},
  '1002': {'id': 1002,
   'text': '<A>% Defense',
   'key_stat': 'defense_percent',
   'vars': [{'key': '<A>',
     'type': 'rand',
     'values': [[1, 3], [2, 4], [3, 5]]}]},
  '1003': {'id': 1003,
   'text': '<A> Defense',
   'key_stat': 'defense_flat',
   '

In [30]:
import random

grades = ["A", "B", "C", "D"]
weights = [0.4, 0.3, 0.2, 0.1]  # Example probability weights for each grade

item_grade = random.choices(grades, weights=weights)[0]
item_grade

'A'

In [32]:
pick_grade = random.choices(list(GRADES.keys()), weights=[grade['weight'] for grade in GRADES.values()])[0]
pick_grade

'Rare'

In [1544]:
# Test if 
common = 0
good = 0
rare = 0
heroic = 0
epic = 0
iters = 100000
for i in range(1,iters):
    pick_grade = random.choices(list(GRADES.keys()), weights=[grade['weight'] for grade in GRADES.values()])[0]
    if pick_grade == 'Normal': common += 1
    elif pick_grade == 'Good' : good += 1
    elif pick_grade == 'Rare': rare += 1
    elif pick_grade == 'Heroic': heroic += 1
    elif pick_grade == 'Epic': epic += 1

In [1545]:
common/iters, uncom/iters, rare/iters, heroic/iters, epic/iters

(0.04914, 0.10013, 0.39992, 0.35088, 0.09854)

In [35]:
GRADES.values()

dict_values([{'starting_substats': 0, 'max_substats': 4, 'weight': 0.05}, {'starting_substats': 1, 'max_substats': 4, 'weight': 0.1}, {'starting_substats': 2, 'max_substats': 4, 'weight': 0.4}, {'starting_substats': 3, 'max_substats': 4, 'weight': 0.35}, {'starting_substats': 4, 'max_substats': 4, 'weight': 0.1}])

In [36]:
weights = [grade['weight'] for grade in GRADES.values()]

In [37]:
weights

[0.05, 0.1, 0.4, 0.35, 0.1]

In [1546]:
GRADES.keys()

dict_keys(['Normal', 'Good', 'Rare', 'Heroic', 'Epic'])

In [39]:
# Get item tier based on level
def get_item_tier(level):
    tier = '' # Initialize Empty tier
    for t in TIERS: # Loop through our tiers dictionary
        if level in TIERS[t]['level_range']: # Check if the level is in the level range in each section
            tier = TIERS[t]['item_tier'] # If the levle matches, then return that tier value
    return tier


In [47]:
TYPES

{'Dagger': {'main_stats': [0], 'sub_stats': [1000, 1001, 1004, 1005, 1006]},
 'Ring': {'main_stats': [0, 1],
  'sub_stats': [1000, 1001, 1002, 1003, 1004, 1005, 1006]}}

In [48]:
TYPES['Ring']['main_stats']

[0, 1]

In [ ]:
z

In [56]:
for id in TYPES['Ring']['main_stats']:
    print(id)

0
1


In [448]:
random.choices(list(GRADES.keys()), weights=[grade['weight'] for grade in GRADES.values()])[0]

'Heroic'

In [460]:
GRADES[random.choices(list(GRADES.keys()), weights=[grade['weight'] for grade in GRADES.values()])[0]]['starting_substats']

3

In [489]:
random.choices(list(GRADES.keys()), weights=[grade['weight'] for grade in GRADES.values()])[0]

'Rare'

In [499]:
GRADES[random.choices(list(GRADES.keys()), weights=[grade['weight'] for grade in GRADES.values()])[0]]['starting_substats']

3

In [1210]:
GRADES['Normal']['starting_substats']

0

In [1211]:
def create_random_item(item_level = None, item_type = None, item_grade = None, item_set = None):

    if item_level is None:
        item_level = random.randint(1, 30)
        item_tier = get_item_tier(item_level)
    else:
        item_tier = get_item_tier(item_level)
    
    if item_type is None:
        item_type = random.choice(list(TYPES))
        
    if item_grade is None:
        item_grade = random.choices(list(GRADES.keys()), weights=[grade['weight'] for grade in GRADES.values()])[0]
        
    if item_set is None:
        item_set = random.choice(list(SETS))

    mainstat = []
    substats = []

    # Get main stat
    mainstat.append(parse_stat(get_random_stat('MainStats', item_type), item_tier))
    
    # Get first substat
    if GRADES[item_grade]['starting_substats'] > 0:
        substats.append(parse_stat(get_random_stat('SubStats', item_type), item_tier))
                       
    # Add more subtats to fill the rest of the item:
    for i in range(1, GRADES[item_grade]['starting_substats']):
        stat = get_non_overlapping_stat(substats, 'SubStats', item_type)
        substats.append(parse_stat(stat, item_tier))

    item = {}
    item['name'] = "VERY VERY VERY BIG ITEM NAME"
    item['type'] = item_type
    item['set'] = item_set
    item['level'] = item_level
    item['tier'] = item_tier
    item['upgrade_level'] = 0
    item['grade'] = item_grade
    item['mainstat'] = mainstat
    item['substats'] = substats

    return item

In [1198]:
def add_new_substat(item):
    if len(item['substats']) < GRADES[item['grade']]['max_substats']:
        substats = item['substats']
        new_stat = get_non_overlapping_stat(substats, 'SubStats', item['type'])
        substats.append(parse_stat(new_stat, item['tier']))
        item['substats'] = substats
    else:
        print('Item already has max number of substats!')
    return item

In [1351]:
item1 = create_random_item(item_grade = 'Epic')
item1

{'name': 'VERY VERY VERY BIG ITEM NAME',
 'type': 'Dagger',
 'set': 'Health',
 'level': 22,
 'tier': 3,
 'upgrade_level': 0,
 'grade': 'Epic',
 'mainstat': [{'id': 0,
   'text': '<A>% Atack',
   'key_stat': 'attack_percent',
   'values': [['<A>', 12]]}],
 'substats': [{'id': 1004,
   'text': '<A> Speed',
   'key_stat': 'speed_flat',
   'values': [['<A>', 4]]},
  {'id': 1001,
   'text': '<A> Atack',
   'key_stat': 'attack_flat',
   'values': [['<A>', 16]]},
  {'id': 1006,
   'text': '<A>% Crit Damage',
   'key_stat': 'crit_damage',
   'values': [['<A>', 16]]},
  {'id': 1000,
   'text': '<A>% Atack',
   'key_stat': 'attack_percent',
   'values': [['<A>', 5]]}]}

In [1177]:
type(GRADES[item1['grade']]['max_substats'])

int

In [1169]:
item1['grade']

'Heroic'

In [1175]:
len(item1['substats'])

1

In [1352]:
add_new_substat(item1)

Item already has max number of substats!


{'name': 'VERY VERY VERY BIG ITEM NAME',
 'type': 'Dagger',
 'set': 'Health',
 'level': 22,
 'tier': 3,
 'upgrade_level': 0,
 'grade': 'Epic',
 'mainstat': [{'id': 0,
   'text': '<A>% Atack',
   'key_stat': 'attack_percent',
   'values': [['<A>', 12]]}],
 'substats': [{'id': 1004,
   'text': '<A> Speed',
   'key_stat': 'speed_flat',
   'values': [['<A>', 4]]},
  {'id': 1001,
   'text': '<A> Atack',
   'key_stat': 'attack_flat',
   'values': [['<A>', 16]]},
  {'id': 1006,
   'text': '<A>% Crit Damage',
   'key_stat': 'crit_damage',
   'values': [['<A>', 16]]},
  {'id': 1000,
   'text': '<A>% Atack',
   'key_stat': 'attack_percent',
   'values': [['<A>', 5]]}]}

In [1526]:
def print_item(item):

    #print(item['name'])
    print(f"+{item['upgrade_level']} {item['grade']} {item['type']} (iLvL {item['level']})")
    print(f"{item['set']} Set")
    # print(f"iLvl: {item['level']}")

    print('---')
    print('MAIN STAT:')
    print(format_stat(item['mainstat'][0]))

    print('---')
    if len(item['substats']) == 1:
        print('SUBSTAT:')
    else:
        print('SUBSTATS:')
    for s in item['substats']:
        print(format_stat(s))

In [990]:
item1 = create_random_item()
print_item(item1)

VERY VERY VERY BIG ITEM NAME
+0 Heroic Dagger (iLvL 24)
Defense Set
---
MAIN STAT:
12% Atack
---
SUBSTATS:
13% Crit Damage
5% Atack
17 Atack


In [961]:
random.choice(list(item1['substats']))['id']

1005

In [1000]:
STATS['SubStats'][str(random.choice(list(item1['substats']))['id'])]

{'id': 1006,
 'text': '<A>% Crit Damage',
 'key_stat': 'crit_damage',
 'vars': [{'key': '<A>',
   'type': 'rand',
   'values': [[10, 13], [12, 14], [13, 16]]}]}

In [1011]:
STATS['SubStats']['1003']['vars'][0]['values'][2][1]

20

In [1002]:
GRADES['Rare']

{'starting_substats': 2, 'max_substats': 4, 'weight': 0.4}

In [1036]:
item1['substats']

[{'id': 1006,
  'text': '<A>% Crit Damage',
  'key_stat': 'crit_damage',
  'values': [['<A>', 13]]},
 {'id': 1000,
  'text': '<A>% Atack',
  'key_stat': 'attack_percent',
  'values': [['<A>', 5]]},
 {'id': 1001,
  'text': '<A> Atack',
  'key_stat': 'attack_flat',
  'values': [['<A>', 17]]}]

In [1075]:
for a in item1['substats']:
    if '1000' in str(a['id']):
        print(a['values'][0][1])

5


In [1026]:
random.randint(1,3)

3

In [1151]:
len(item2['substats'])

3

In [1200]:
GRADES

{'Normal': {'starting_substats': 0, 'max_substats': 4, 'weight': 0.05},
 'Uncommon': {'starting_substats': 1, 'max_substats': 4, 'weight': 0.1},
 'Rare': {'starting_substats': 2, 'max_substats': 4, 'weight': 0.4},
 'Heroic': {'starting_substats': 3, 'max_substats': 4, 'weight': 0.35},
 'Epic': {'starting_substats': 4, 'max_substats': 4, 'weight': 0.1}}

In [1442]:
def upgrade_main_stat(item):
    main_stat_id = str(item['mainstat'][0]['id'])
    upgrade_level = item['upgrade_level']
    
    if main_stat_id in STATS['MainStats']:
        upgrade_val = STATS['MainStats'][main_stat_id]['upgrade_vals'][upgrade_level]
        item['mainstat'][0]['values'][0][1] += upgrade_val
        return True
    return False


def upgrade_random_sub_stat(item):
    if 'substats' in item and item['substats']:
        random_substat = random.choice(item['substats'])
        random_substat_id = str(random_substat['id'])
        
        if random_substat_id in STATS['SubStats']:
            upgrade_val_range = STATS['SubStats'][random_substat_id]['vars'][0]['values'][item['tier'] - 1]
            upgrade_val_min, upgrade_val_max = upgrade_val_range
            upgrade_val = random.randint(upgrade_val_min, upgrade_val_max)
            
            for s in item['substats']:
                if random_substat_id in str(s['id']):
                    s['values'][0][1] += upgrade_val
                    return True
    return False


def upgrade_item(item):
    upgrade_level = item['upgrade_level']
    if upgrade_level < 5:
        
        # Upgrade Main Stat
        upgrade_main_stat(item)
        
        # Normal Gear:
        if item['grade'] == 'Normal' and upgrade_level < 4:
            add_new_substat(item)
            
        # Good Gear:
        elif item['grade'] == 'Good' and upgrade_level < 1:
            upgrade_random_sub_stat(item)
        elif item['grade'] == 'Good' and upgrade_level < 4:
            add_new_substat(item)
            
        # Rare Gear:
        elif item['grade'] == 'Rare' and upgrade_level < 2:
            upgrade_random_sub_stat(item)
        elif item['grade'] == 'Rare' and upgrade_level < 4:
            add_new_substat(item)

        # Heroic Gear:
        elif item['grade'] == 'Heroic' and upgrade_level < 3:
            upgrade_random_sub_stat(item)
        elif item['grade'] == 'Heroic' and upgrade_level < 4:
            add_new_substat(item)
        
        # For Epic Gear as well as other gear at upgrade level 4~
        else:
            upgrade_random_sub_stat(item)
        
        # Increase upgrade level by 1
        item['upgrade_level'] += 1
        
        return item
    
    else:
        print('Item already at max upgrade level!')
        return item

In [1554]:
item2 = create_random_item()
print_item(item2)

+0 Rare Ring (iLvL 5)
Health Set
---
MAIN STAT:
50 Atack
---
SUBSTATS:
3% Atack
1% Crit Rate Chance


In [1559]:
print_item(upgrade_item(item2))

+5 Rare Ring (iLvL 5)
Health Set
---
MAIN STAT:
160 Atack
---
SUBSTATS:
8% Atack
3% Crit Rate Chance
3 Speed
14 Atack


In [1541]:
item2

{'name': 'VERY VERY VERY BIG ITEM NAME',
 'type': 'Ring',
 'set': 'Speed',
 'level': 23,
 'tier': 3,
 'upgrade_level': 4,
 'grade': 'Heroic',
 'mainstat': [{'id': 0,
   'text': '<A>% Atack',
   'key_stat': 'attack_percent',
   'values': [['<A>', 34]]}],
 'substats': [{'id': 1005,
   'text': '<A>% Crit Rate Chance',
   'key_stat': 'crit_rate',
   'values': [['<A>', 7]]},
  {'id': 1001,
   'text': '<A> Atack',
   'key_stat': 'attack_flat',
   'values': [['<A>', 32]]},
  {'id': 1000,
   'text': '<A>% Atack',
   'key_stat': 'attack_percent',
   'values': [['<A>', 9]]},
  {'id': 1004,
   'text': '<A> Speed',
   'key_stat': 'speed_flat',
   'values': [['<A>', 2]]}]}

In [591]:
def has_mainstat(item, ids = []):

	mainstat = []
	mainstat.extend(item['mainstat'])

	list_ids = [m['id'] for m in mainstat]

	for id in ids:
		if id not in list_ids:
			return False 
	return True

In [592]:
def has_substats(item, ids = []):

	substats = []
	substats.extend(item['substats'])

	list_ids = [s['id'] for s in substats]

	for id in ids:
		if id not in list_ids:
			return False 
	return True

In [15]:
# def to_view(item):

# 	view_dict = {}
# 	view_dict['name'] = item['name']
# 	view_dict['tier'] = item['tier']
# 	view_dict['type'] = item['type']
# 	view_dict['stats'] = []
# 	view_dict['affixes'] = []
# 	view_dict['legendary'] = []
# 	view_dict['sockets'] = []

# 	for stat in item['stats']:
# 		view_dict['stats'].append(format_affix(stat))
# 	for affix in item['affixes']:
# 		view_dict['affixes'].append(format_affix(affix))
# 	for affix in item['legendary']:
# 		view_dict['legendary'].append(format_affix(affix))
# 	for socket in item['sockets']:
# 		view_dict['sockets'].append(format_affix(socket))

# 	view_dict['main_stat'] = view_dict['stats'].pop(0) #temp
# 	view_dict['sell_value'] = 999
# 	view_dict['durability'] = 100

# 	dmg_icon = 'default'
# 	if item['type'] in CONFIG['offensive']:
# 		dmg_icon = 'offensive'
# 	elif item['type'] in CONFIG['defensive']:
# 		dmg_icon = 'defensive'
# 	view_dict['dmg_icon'] = dmg_icon

# 	return view_dict
